## Import dependencies

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer
import numpy as np
from scipy.fftpack import dct
import numpy as np
import tensorflow.keras as K
import tensorflow.keras.backend as Kback
!pip install tensorflow-wavelets
import tensorflow_wavelets.Layers.DWT as DWT

## Dataloader

In [ ]:
train_datagen = K.preprocessing.image.ImageDataGenerator(rescale = 1./255)   

train_dataset  = train_datagen.flow_from_directory(directory = '/kaggle/input/ham10000-data/HAM10000_DATA/train_dir',
                                                   target_size = (256,256),
                                                   class_mode = 'categorical',
                                                   subset = 'training',
                                                   shuffle=True,
                                                   batch_size = 64)
validation_dataset  = train_datagen.flow_from_directory(directory = '/kaggle/input/ham10000-data/HAM10000_DATA/train_dir',
                                                   target_size = (256,256),
                                                   class_mode = 'categorical',
                                                   subset = 'training',
                                                   shuffle=True,
                                                   batch_size = 64)


test_datagen = K.preprocessing.image.ImageDataGenerator(rescale = 1./255)   

test_dataset  = test_datagen.flow_from_directory(directory = '/kaggle/input/ham10000-data/HAM10000_DATA/test_dir',
                                                   target_size = (256,256),
                                                   class_mode = 'categorical',
                                                   subset = 'training',
                                                   shuffle=False,
                                                   batch_size = 64)

In [ ]:
print(train_dataset.class_indices)
print(validation_dataset.class_indices)
print(test_dataset.class_indices)

## Metrics

In [ ]:
def f1_score(y_true, y_pred):
    true_positives = Kback.sum(Kback.round(Kback.clip(y_true * y_pred, 0, 1)))
    possible_positives = Kback.sum(Kback.round(Kback.clip(y_true, 0, 1)))
    predicted_positives = Kback.sum(Kback.round(Kback.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + Kback.epsilon())
    recall = true_positives / (possible_positives + Kback.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+Kback.epsilon())
    return f1_val

METRICS = [
      "accuracy",
      K.metrics.Precision(name='precision'),
      K.metrics.Recall(name='recall'),
      K.metrics.AUC(name='auc'),
      f1_score
]

## Model

#### FFT

In [ ]:
def fft_2d(feature_map):
    feature_map = tf.cast(feature_map, tf.complex64)
    X1 = tf.signal.fft2d(feature_map)
    X1 = tf.abs(X1)
    return X1 

#### SaFA

In [ ]:
def Similarity(x):
    batch,H,W,C = x.shape
    x = K.layers.Reshape((H, W))(x)
    x_bar = K.layers.Permute((2, 1))(x)
    sim = K.layers.Dot(axes=(1,2), normalize=True)([x,x_bar])
    sim = tf.expand_dims(sim, axis=-1)
    return sim

def FeatureChange(x):
    batch,H,W,C = x.shape
    x_h = K.layers.Reshape((H, W))(x)
    lstm_h,_,_ = K.layers.LSTM(H, return_sequences=True, return_state=True)(x_h)
    x_w = K.layers.Permute((2, 1))(x_h)
    lstm_w,_,_ = K.layers.LSTM(W, return_sequences=True, return_state=True)(x_w)
    print(lstm_w.shape)
    lstm = lstm_h+lstm_w
    lstm = tf.expand_dims(lstm, axis=-1)
    return lstm

def SaFA(inputs):
    x = K.layers.SeparableConv2D(256, 5, padding="same")(inputs)
    x = K.layers.SeparableConv2D(64, 3, padding="same")(x)
    x = K.layers.SeparableConv2D(1, 1, padding="same")(x)
    lstm = FeatureChange(x)
    sim = Similarity(x)
    attn = sim*inputs
    x = K.layers.SeparableConv2D(256, 1, strides=(2, 2), padding="same")(attn)
    x = K.layers.SeparableConv2D(64, 3, padding="same")(x)
    x = K.layers.SeparableConv2D(1, 1, padding="same", activation='sigmoid')(x)
    inputs = K.layers.MaxPooling2D(pool_size=(2, 2),padding="same")(inputs)
    x = inputs*x
    return x

#### Gradients

In [ ]:
class Gradients(K.layers.Layer):
    def call(self, inputs):
        alpha = inputs
        gradients_alpha = tf.gradients(alpha, [alpha])[0]
        a = tf.reduce_mean(gradients_alpha, axis=[-1,-2,-3], keepdims=True)
        return a

#### Soft Attention

In [ ]:
class SoftAttention(K.layers.Layer):
    def __init__(self,ch,m,concat_with_x=False,aggregate=False,**kwargs):
        self.channels=int(ch)
        self.multiheads = m
        self.aggregate_channels = aggregate
        self.concat_input_with_scaled = concat_with_x

        
        super(SoftAttention,self).__init__(**kwargs)

    def build(self,input_shape):

        self.i_shape = input_shape

        kernel_shape_conv3d = (self.channels, 3, 3) + (1, self.multiheads) # DHWC
    
        self.out_attention_maps_shape = input_shape[0:1]+(self.multiheads,)+input_shape[1:-1]
        
        if self.aggregate_channels==False:

            self.out_features_shape = input_shape[:-1]+(input_shape[-1]+(input_shape[-1]*self.multiheads),)
        else:
            if self.concat_input_with_scaled:
                self.out_features_shape = input_shape[:-1]+(input_shape[-1]*2,)
            else:
                self.out_features_shape = input_shape
        

        self.kernel_conv3d = self.add_weight(shape=kernel_shape_conv3d,
                                        initializer='he_uniform',
                                        name='kernel_conv3d')
        self.bias_conv3d = self.add_weight(shape=(self.multiheads,),
                                      initializer='zeros',
                                      name='bias_conv3d')

        super(SoftAttention, self).build(input_shape)

    def call(self, x):

        exp_x = Kback.expand_dims(x,axis=-1)

        c3d = Kback.conv3d(exp_x,
                     kernel=self.kernel_conv3d,
                     strides=(1,1,self.i_shape[-1]), padding='same', data_format='channels_last')
        conv3d = Kback.bias_add(c3d,
                        self.bias_conv3d)
        conv3d = K.layers.Activation('relu')(conv3d)

        conv3d = Kback.permute_dimensions(conv3d,pattern=(0,4,1,2,3))

        
        conv3d = Kback.squeeze(conv3d, axis=-1)
        conv3d = Kback.reshape(conv3d,shape=(-1, self.multiheads ,self.i_shape[1]*self.i_shape[2]))

        softmax_alpha = Kback.softmax(conv3d, axis=-1) 
        softmax_alpha = K.layers.Reshape(target_shape=(self.multiheads, self.i_shape[1],self.i_shape[2]))(softmax_alpha)

        
        if self.aggregate_channels==False:
            exp_softmax_alpha = Kback.expand_dims(softmax_alpha, axis=-1)       
            exp_softmax_alpha = Kback.permute_dimensions(exp_softmax_alpha,pattern=(0,2,3,1,4))
   
            x_exp = Kback.expand_dims(x,axis=-2)
   
            u = K.layers.Multiply()([exp_softmax_alpha, x_exp])   
  
            u = K.layers.Reshape(target_shape=(self.i_shape[1],self.i_shape[2],u.shape[-1]*u.shape[-2]))(u)

        else:
            exp_softmax_alpha = Kback.permute_dimensions(softmax_alpha,pattern=(0,2,3,1))

            exp_softmax_alpha = Kback.sum(exp_softmax_alpha,axis=-1)

            exp_softmax_alpha = Kback.expand_dims(exp_softmax_alpha, axis=-1)

            u = K.layers.Multiply()([exp_softmax_alpha, x])   

        if self.concat_input_with_scaled:
            o = K.layers.Concatenate(axis=-1)([u,x])
        else:
            o = u
        
        return [o, softmax_alpha]

    def compute_output_shape(self, input_shape): 
        return [self.out_features_shape, self.out_attention_maps_shape]

    
    def get_config(self):
        return super(SoftAttention,self).get_config()

#### Deep Learner

In [ ]:
input_layer = K.Input(shape=(256,256,3))
deep_learner = K.applications.DenseNet121(include_top = False, weights = "imagenet", input_tensor = input_layer)
for layer in deep_learner.layers:
    layer.trainable = True
# for i, layer in enumerate(deep_learner.layers):
#     print(i, layer.name, "-", layer.trainable)

#### Model

In [ ]:
input_img = K.layers.Input(shape=(256,256,3)) 
feat_img = deep_learner(input_img)
wav = DWT.DWT(name="haar",concat=0)(feat_img)
wav = K.layers.SeparableConv2D(1024, 1, padding="same")(wav)
attention_layer,map2 = SoftAttention(aggregate=True,m=16,concat_with_x=False,ch=int(feat_img.shape[-1]),name='soft_attention')(feat_img)
attention_layer = K.layers.MaxPooling2D(pool_size=(2, 2),padding="same")(attention_layer)
#conv = K.layers.MaxPooling2D(pool_size=(2, 2),padding="same")(feat_img)

grad_attn = Gradients()(attention_layer)
grad_wav = Gradients()(wav)
grad_attn = 1-(grad_attn/(grad_attn+grad_wav))
grad_wav = 1-(grad_wav/(grad_attn+grad_wav))
attention_layer = grad_attn*attention_layer+grad_wav*wav
conv = K.layers.concatenate([SaFA(feat_img),attention_layer])
conv  = K.layers.Activation('relu')(conv)
flat = K.layers.GlobalAveragePooling2D()(conv)
output = K.layers.Dense(7, activation='softmax')(flat)

model = K.Model(inputs=input_img, outputs=output)
optimizer = K.optimizers.Adam(lr=0.01)
model.compile(loss=["categorical_crossentropy"], metrics=METRICS, optimizer = optimizer)
model.summary()

## Training

In [ ]:
model_checkpoint_callback = K.callbacks.ModelCheckpoint(
    filepath='densenet121_SA.hdf5',
    monitor='val_f1_score',
    save_best_only=True,
    save_weights_only=True,
    mode='max',
    verbose=1
    )

history = model.fit(train_dataset,
                    epochs = 25,
                    validation_data = validation_dataset,
                    verbose = 1,
                    callbacks=[model_checkpoint_callback],
                    shuffle = True)

#### Training plots

In [ ]:
import matplotlib.pyplot as plt

def Train_Val_Plot(acc, val_acc, loss, val_loss, auc, val_auc, precision, val_precision, recall, val_recall, f1_score, val_f1_score):
    fig, axes = plt.subplots(2, 3, figsize=(20, 10))
    fig.suptitle("MODEL'S METRICS VISUALIZATION")

    axes[0, 0].plot(range(1, len(acc) + 1), acc)
    axes[0, 0].plot(range(1, len(val_acc) + 1), val_acc)
    axes[0, 0].set_title('History of Accuracy')
    axes[0, 0].set_xlabel('Epochs')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend(['training', 'validation'])

    axes[0, 1].plot(range(1, len(loss) + 1), loss)
    axes[0, 1].plot(range(1, len(val_loss) + 1), val_loss)
    axes[0, 1].set_title('History of Loss')
    axes[0, 1].set_xlabel('Epochs')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend(['training', 'validation'])

    axes[0, 2].plot(range(1, len(auc) + 1), auc)
    axes[0, 2].plot(range(1, len(val_auc) + 1), val_auc)
    axes[0, 2].set_title('History of AUC')
    axes[0, 2].set_xlabel('Epochs')
    axes[0, 2].set_ylabel('AUC')
    axes[0, 2].legend(['training', 'validation'])

    axes[1, 0].plot(range(1, len(precision) + 1), precision)
    axes[1, 0].plot(range(1, len(val_precision) + 1), val_precision)
    axes[1, 0].set_title('History of Precision')
    axes[1, 0].set_xlabel('Epochs')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].legend(['training', 'validation'])

    axes[1, 1].plot(range(1, len(recall) + 1), recall)
    axes[1, 1].plot(range(1, len(val_recall) + 1), val_recall)
    axes[1, 1].set_title('History of Recall')
    axes[1, 1].set_xlabel('Epochs')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].legend(['training', 'validation'])

    axes[1, 2].plot(range(1, len(f1_score) + 1), f1_score)
    axes[1, 2].plot(range(1, len(val_f1_score) + 1), val_f1_score)
    axes[1, 2].set_title('History of F1 score')
    axes[1, 2].set_xlabel('Epochs')
    axes[1, 2].set_ylabel('Recall')  # Corrected from 'Recall' to 'F1 score'
    axes[1, 2].legend(['training', 'validation'])

    plt.tight_layout()
    plt.show()

# Call the function with your history data
Train_Val_Plot(history.history['accuracy'], history.history['val_accuracy'],
               history.history['loss'], history.history['val_loss'],
               history.history['auc'], history.history['val_auc'],
               history.history['precision'], history.history['val_precision'],
               history.history['recall'], history.history['val_recall'],
               history.history['f1_score'], history.history['val_f1_score'])

## Testing

In [ ]:
model.load_weights("/kaggle/working/densenet121_SA.hdf5")

# Evaluate the model
loss, accuracy, precision, recall, auc, f1_score = model.evaluate(test_dataset)
print("Accuracy", accuracy)
print("Loss", loss)
print("Precision", precision)
print("Recall", recall)
print("AUC", auc)
print("F1-score", f1_score)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
Y_pred = model.predict_generator(test_dataset, 1157)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
disp = ConfusionMatrixDisplay(confusion_matrix(test_dataset.classes, y_pred),display_labels=['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc'])
disp.plot()
plt.show()
print('Classification Report')
target_names = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
print(classification_report(test_dataset.classes, y_pred, target_names=target_names))